In [21]:
import numba
from numba import jit, guvectorize, float64, generated_jit

@jit(nopython=True)
def scalar_interp_1d(u,x,y):
    K_0 = x.shape[0]
    ii0 = np.searchsorted(x,u)
    i0 = np.maximum(0, np.minimum(ii0-1, K_0-2))
    x0, x1 = (x[i0], x[i0+1])
    y0, y1 = (y[i0], y[i0+1])
    l0, = ((u-x0)/(x1-x0), ) # barycentric coordinate
    res = y0 + l0*(y1-y0)
    return res


@jit(nopython=True)
def vec_interp_1d(u,x,y):
    N = u.shape[0]
    out = np.zeros(N)
    for n in range(N):
        out[n] = scalar_interp_1d(u[n], x, y)
    return out


@generated_jit
def interp_1d(u,x,y):
    if isinstance(u,  numba.types.Array) and u.ndim == 1:
        return vec_interp_1d
    elif isinstance(u, numba.types.Float):
        return scalar_interp_1d
    else:
        raise Exception(f"Unsupported type for u: {u}.")

In [23]:
import numpy as np

In [24]:
x = np.linspace(-4, 4, 10)
y = np.cos(x)
interp_1d(0.5, x, y)

0.8611239133554044

In [25]:
u = np.linspace(-4, 4, 20)
interp_1d(u, x, y)

array([-0.65364362, -0.81748713, -0.98133064, -0.83397145, -0.64771193,
       -0.29627306,  0.10235992,  0.44606245,  0.76229975,  0.90284967,
        0.90284967,  0.76229975,  0.44606245,  0.10235992, -0.29627306,
       -0.64771193, -0.83397145, -0.98133064, -0.81748713, -0.65364362])